<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import dependencies
import pandas as pd
import numpy as np
import random
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# Set the column width
pd.set_option('max_colwidth', 200)

In [11]:
import gdown
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


#URL for downloading lung cancer data
url_lc = 'https://drive.google.com/uc?id=1dCjysGH7CjseeifERUin9gaoKchtcGnh'
gdown.download(url_lc, 'news_article.csv', quiet=False)
# https://drive.google.com/file/d/1dCjysGH7CjseeifERUin9gaoKchtcGnh/view?usp=drive_link


#Read the Lung cancer data.csv
news_articles_df = pd.read_csv(url_lc)
news_articles_df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading...
From: https://drive.google.com/uc?id=1dCjysGH7CjseeifERUin9gaoKchtcGnh
To: /content/news_article.csv
100%|██████████| 1.43M/1.43M [00:00<00:00, 105MB/s]


,headline
0,Is 22 Too Young To Marry A 36-Year-Old? 'The Bachelor' Investigates
1,The Only Shopping Guide For Cyber Monday You Need
2,Taylor Swift Dances When No One Can See Her In New 'Delicate' Video
3,How To Say 'Cheers' In 20 Languages (AUDIO)
4,'Welcome To Hell': Rio Police Warn They Can't Promise Olympic Protection


In [12]:
# Load the news_articles.csv into a DataFrame.
# news_articles_df = pd.read_csv('Resources/news_articles.csv')
# Display the first 20 headlines
news_articles_df.head(10)

,headline
0,Is 22 Too Young To Marry A 36-Year-Old? 'The Bachelor' Investigates
1,The Only Shopping Guide For Cyber Monday You Need
2,Taylor Swift Dances When No One Can See Her In New 'Delicate' Video
3,How To Say 'Cheers' In 20 Languages (AUDIO)
4,'Welcome To Hell': Rio Police Warn They Can't Promise Olympic Protection
5,Conservative Pundit Points Out Where Real Blame For GOP’s ‘Descent Into Madness’ Lies
6,We Asked The American Public To Settle 5 Of The Internet's Dumbest Debates
7,'Teen Mom OG's' Catelynn Lowell Heads To Treatment Over Suicidal Thoughts
8,The Major Problem With Electric Cars | TIME.com
9,Why Is Nobel-Winning Economist Richard Thaler So Jovial?


## Preprocess the Text

In [13]:
# Get the info on the DataFrame
news_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23377 entries, 0 to 23376
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  23376 non-null  object
dtypes: object(1)
memory usage: 182.8+ KB


In [16]:
# Remove digits and non-alphabetic characters
#lamba func is 2 parts , lamba variable : expression
news_articles_df["headline"] = news_articles_df["headline"].apply(lambda x: re.sub(r"[^a-zA-Z\s ]","",str(x)))
news_articles_df.head()

,headline
0,Is Too Young To Marry A YearOld The Bachelor Investigates
1,The Only Shopping Guide For Cyber Monday You Need
2,Taylor Swift Dances When No One Can See Her In New Delicate Video
3,How To Say Cheers In Languages AUDIO
4,Welcome To Hell Rio Police Warn They Cant Promise Olympic Protection


## Process the Text to Tokens and Counts.

In [18]:
# Create an instance of the CountVectorizer and set the max_df to 0.95 and min_df to 10, and use the "english" stopwords.
#max_df = remove words that appear more than 95% of the word
#min_df = word must appear in atleast 10 documents to be included
cv = CountVectorizer(max_df=0.95,min_df=10, stop_words='english')
cv

CountVectorizer(max_df=0.95, min_df=10, stop_words='english')

In [20]:
# Get the headlines.
headlines = news_articles_df["headline"]
headlines

,headline
0,Is Too Young To Marry A YearOld The Bachelor Investigates
1,The Only Shopping Guide For Cyber Monday You Need
2,Taylor Swift Dances When No One Can See Her In New Delicate Video
3,How To Say Cheers In Languages AUDIO
4,Welcome To Hell Rio Police Warn They Cant Promise Olympic Protection
...,...
23372,Bidens Health Agenda Starts With Reversing Everything Trump Did In The Last Years
23373,You Know Where You Are From the Very First Bite
23374,Cheeses We Would Happily Marry If That Was Allowed
23375,Donald Trump Has A Surprising Response To Golfer Rory McIlroys Criticism


In [21]:
# Transform each row from the headlines Series to a DTM.
#DTM - Document Term Matrix . How much a specific term appears in the entire collection of documents
dtm = cv.fit_transform(headlines)
# Get the shape of the DTM.
print(dtm.shape)
#rows and columns

(23377, 3149)


In [22]:
# Get the length of the vocabulary
len(cv.get_feature_names_out())

3149

In [23]:
# Look at 100 random words in the vocabulary
print(cv.get_feature_names_out()[:100])

['aaron' 'abandoned' 'ability' 'able' 'abortion' 'abroad' 'absolutely'
 'abuse' 'abused' 'access' 'accident' 'accidental' 'accidentally'
 'according' 'account' 'accounts' 'accusations' 'accused' 'accuser'
 'accusers' 'accuses' 'act' 'acting' 'action' 'activists' 'actor'
 'actress' 'actually' 'ad' 'adam' 'add' 'added' 'adding' 'address' 'adds'
 'adele' 'administration' 'admits' 'adorable' 'ads' 'adults' 'adventure'
 'adventures' 'advice' 'adviser' 'advocates' 'affleck' 'afford'
 'affordable' 'afghanistan' 'africa' 'age' 'agency' 'agenda' 'agent' 'ago'
 'agree' 'agrees' 'ahead' 'aid' 'aide' 'aides' 'aim' 'aims' 'air' 'airbnb'
 'airline' 'airlines' 'airplane' 'airport' 'airports' 'al' 'alabama'
 'alaska' 'album' 'alec' 'alex' 'alexandria' 'ali' 'alive' 'allegations'
 'alleged' 'allegedly' 'allen' 'allies' 'allow' 'allowed' 'allstar'
 'alternative' 'amazing' 'amazon' 'amazons' 'ambassador' 'amber' 'america'
 'american' 'americans' 'americas' 'amid' 'amy']


In [24]:
# Print the first 500 elements (transformed words)from the 1st row, i.e., document.
#converting the matrix to array to print it out
print(dtm.toarray()[0][:500])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [27]:
# Get the feature names (words) from the CountVectorizer
feature_names = cv.get_feature_names_out()

# Get all the non-zero elements from the first row.
non_zero_elements = dtm.toarray()[0]

# Get the indices for each non-zero element.
non_zero_indices = non_zero_elements.nonzero()[0]

# Print out the word and the number of times the word is in the row.
for i in non_zero_indices:
   print(f"The word : {feature_names[i]} - index : {i} - count : {non_zero_elements[i]}")


The word : bachelor - index : 183 - count : 1
The word : yearold - index : 3131 - count : 1
The word : young - index : 3138 - count : 1


In [31]:
# Convert the DTM to a DataFrame

dtm_df = pd.DataFrame(dtm.toarray(),columns=cv.get_feature_names_out())
# Display some random columns and the first 20 rows of the DataFrame.
dtm_df.iloc[:,133:149:].head(20)

,argentina,argument,ariana,arizona,army,arrest,arrested,art,article,artificial,ashley,asian,ask,asked,asking,asks
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## LDA

In [32]:
# Create and instance of the LatentDirichletAllocation() class with 7 topics.
#business, politics, entertainment, food and drink, sports, technology and travel
LDA = LatentDirichletAllocation(n_components=7, random_state = 45)
# Fit the model with our DTM data. This may take awhile if you have a large amount of documents.
LDA_data = LDA.fit(dtm)

In [33]:
# Get the values of each topic-word distribution.
topic_word_distributions = LDA_data.components_
print(topic_word_distributions)

[[ 6.68868359  0.1428575   1.16038864 ...  0.14351059  0.1428573
  25.14191545]
 [ 0.14331542  0.14300083  0.14298379 ...  0.14328801  0.14286797
   0.14295194]
 [ 0.14297809  2.89817877  3.14252277 ...  0.14302848  0.14285735
   0.14287649]
 ...
 [ 0.14292072  0.14289515  0.14291571 ...  1.6273334   0.1429087
   0.1429782 ]
 [ 0.14294079  0.14308772  0.14307667 ...  5.61167278 10.14258776
   0.14323906]
 [16.59615124 12.38685925  0.14297134 ...  5.1882435   0.14300355
   0.14301934]]


In [ ]:
# Get the length of the array of each topic. It should be the same as the vocabulary.


In [ ]:
# Get the array of the first topic

# This is the ranking of each word in the array. Lower values have less impact than higher values.


In [ ]:
# Get the indices for the first topic in descending order.


# Use the sorted indices to the values from greatest to least.


## Using `argsort()`
---
- `argsort()` returns index positions from least to greatest.

In [ ]:
# Define an array of values index 0 = 10, index 1 = 200, index 2 = 1.
arr = np.array([10, 200, 1])
# Print out the indices after sorting the array from least to greatest, i.e., 1, 10, 200:
print(f"The indices the the array, '10, 200, 1' from least to greatest: {np.argsort(arr)}")
# Reverse the sort from greatest to least.
print(f"The indices the the array, '10, 200, 1' from greatest to least: {np.argsort(-arr)}")

In [ ]:
# Sort the array of the first topic


In [ ]:
# Get the value of the word that is least representative of this topic
print(f"The value of the word that is least representative of this topic is: {first_topic[1716]}")
# Get the value of the word that is most representative of this topic
print(f"The value of the word that is most representative of this topic is: {first_topic[1688]}")

In [ ]:
# Get the indices of the top ten words for the first topic (e.g., top 10 words for topic 0):


In [ ]:
# Get the top ten words from the indices.


In [ ]:
# Get the bottom ten words from the indices.


In [ ]:
# Print the top 20 words for each topic


### Taking our best guess at the topics.
---
- TOPIC 1: **Travel**
- TOPIC 2: **Sports**
- TOPIC 3: **Food**
- TOPIC 4: **Politics**
- TOPIC 5: **Business**
- TOPIC 6: **Entertainment**
- TOPIC 7: **Technology**

### Assigning the Topic to the Headline

In [ ]:
# Transform our DTM so we get an array with the (number_of_documents, number_of_topics).
topic_results = LDA.transform(dtm)

# Get the shape of the topic results
topic_results.shape

In [ ]:
# Get the first headline's topic probability distribution rounded to 6 decimal places.


In [ ]:
# Get the sorted indices for each topic in the first headline.

# Print the ranking of topics for the headline


In [ ]:
# Get the topic with the highest probability.


This means that our model thinks that the first article belongs to topic "2".

In [ ]:
# Read in our original news headlines.

# Combine the original data with the topic label.
news_articles_df_2['topic'] = (topic_results.argmax(axis=1)+1)

In [ ]:
# Get the first 20 rows.


In [ ]:
# Get the last 20 rows.
